In [ ]:
import matlab
import matlab.engine

eng=matlab.engine.start_matlab()
print(eng.isprime(7))

In [ ]:
eng.test(nargout=0)

In [ ]:
# 读取matlab 工作区变量
eng.eval('x')

In [ ]:
# 在matlab中写入变量
eng.workspace['py']=1
print(eng.eval('py'))

In [ ]:
import matlab
import matlab.engine
import time
import numpy as np
import matplotlib.pyplot as plt

eng = matlab.engine.start_matlab()
env_name = 't'
eng.load_system(env_name)

eng.set_param('t/gain', 'value', str(0), nargout=0) # 初始化控制量u为0
eng.set_param('t/pause_time', 'value', str(1), nargout=0) # 初始化第一个内部暂停时间为1s
eng.set_param(env_name, 'StopTime', str(5), nargout=0) # 设定仿真截止时间
eng.set_param(env_name, 'SimulationCommand', 'start', nargout=0) # 开启控制，会在1.1s自动pause
# 此处应该在1.1s时pause, 然后修改pause_time为2
t = np.array(eng.eval('out.tout')).reshape(-1)

for i1 in range(2,6,1):
    eng.set_param('t/gain', 'value', str(i1), nargout=0)
    eng.set_param('t/pause_time', 'value', str(i1), nargout=0)
    eng.set_param(env_name, 'SimulationCommand', 'continue', nargout=0)
    t = np.array(eng.eval('out.t.Data')).reshape(-1)


eng.set_param(env_name, 'SimulationCommand', 'stop', nargout=0)
t = np.array(eng.eval('out.tout'))
y = np.array(eng.eval('out.y.Data'))
eng.quit()
fig = plt.figure(figsize=(5, 5))
plt.plot(t, y, 'or-')
plt.show()

In [ ]:
import matlab
import matlab.engine
import time
import numpy as np
import matplotlib.pyplot as plt

class MyAgent:

    def __init__(self,simName,simParam):
        eng = matlab.engine.start_matlab()
        self.eng=eng
        self.env_name=simName
        self.eng.load_system(self.env_name)
        self.simParam=simParam

    def reset(self,init_pause_time=0,init_action=0):
        self.done=False
        self.eng.set_param(self.env_name + '/input', 'value', str(init_action), nargout=0)  # initial control signal
        self.eng.set_param(self.env_name, 'StopTime', str(self.simParam['stoptime']), nargout=0) 
        self.eng.set_param(self.env_name + '/pause_time', 'value', str(init_pause_time), nargout=0)  
        self.eng.set_param(self.env_name, 'SimulationCommand', 'start', nargout=0) # 开启控制     
        #self.state = np.array(self.eng.eval('out.y.Data'))[-1]        
        # 设定初始的pause_time
        #return self.state
    
    def stop(self):
        self.eng.set_param(self.env_name, 'SimulationCommand', 'stop', nargout=0)
        t = np.array(self.eng.eval('out.tout'))
        y = np.array(self.eng.eval('out.y.Data'))
        fig = plt.figure(figsize=(5, 5))
        print(t[1:10])
        plt.plot(t, y)
        plt.show()
        self.eng.quit()

    def simstep(self,action,simpleTime):
        self.model_status = self.eng.get_param(self.env_name, 'SimulationStatus')
        i=0
        while True:
            if self.model_status == 'paused':
                self.eng.set_param(self.env_name + '/input', 'value', str(action), nargout=0)  # initial control signal
                self.eng.set_param(self.env_name + '/pause_time', 'value', str(simpleTime), nargout=0)
                self.eng.set_param(self.env_name, 'SimulationCommand', 'continue', nargout=0)

                i=0
                while True:
                    if self.model_status == 'paused':
                        self.clock = np.array(self.eng.eval('out.tout'))[-1]
                        self.next_state = np.array(self.eng.eval('out.y.Data'))[-1]
#                        self.reward = np.array(self.eng.eval('out.reward.Data'))[-1]

                        return self.clock, self.next_state
                    else:
                        i=i+1
                        if i>10:
                            print('error2');
                            break
            else:
                i=i+1
                if i>10:
                    print('error1');
                    break

simParam={
    'stoptime':10,
    'pause_time':0,
}

mandoAgent=MyAgent('py_ma',simParam)

mandoAgent.reset()
stime=0.02
num=int(simParam['stoptime']/stime)

for ii in range(1,num): 
    if ii==1:
        action=0
    else:
        action=-2*y
    t,y=mandoAgent.simstep(action,ii*stime)

mandoAgent.stop()


###############################

the first DQN with MATLAB for pendulum

In [ ]:
import matlab
import matlab.engine
import time
import numpy as np
import matplotlib.pyplot as plt

class MyAgent:

    def __init__(self,simName,simParam):
        eng = matlab.engine.start_matlab()
        self.eng=eng
        self.env_name=simName
        self.eng.load_system(self.env_name)
        self.simParam=simParam

    def reset(self,init_pause_time=0,init_action=0):
        self.done=False
        self.eng.set_param(self.env_name + '/input', 'value', str(init_action), nargout=0)  # initial control signal
        self.eng.set_param(self.env_name, 'StopTime', str(self.simParam['stoptime']), nargout=0) 
        self.eng.set_param(self.env_name + '/pause_time', 'value', str(init_pause_time), nargout=0)  
        self.eng.set_param(self.env_name, 'SimulationCommand', 'start', nargout=0) # 开启控制     
        #self.state = np.array(self.eng.eval('out.y.Data'))[-1]        
        # 设定初始的pause_time
        #return self.state
    
    def stop(self):
        self.eng.set_param(self.env_name, 'SimulationCommand', 'stop', nargout=0)
        t = np.array(self.eng.eval('out.tout'))
        y = np.array(self.eng.eval('out.y.Data'))
        fig = plt.figure(figsize=(5, 5))
        print(t[1:10])
        plt.plot(t, y)
        plt.show()
        self.eng.quit()

    def simstep(self,action,simpleTime):
        self.model_status = self.eng.get_param(self.env_name, 'SimulationStatus')
        i=0
        while True:
            if self.model_status == 'paused':
                self.eng.set_param(self.env_name + '/input', 'value', str(action), nargout=0)  # initial control signal
                self.eng.set_param(self.env_name + '/pause_time', 'value', str(simpleTime), nargout=0)
                self.eng.set_param(self.env_name, 'SimulationCommand', 'continue', nargout=0)

                i=0
                while True:
                    if self.model_status == 'paused':
                        self.clock = np.array(self.eng.eval('out.tout'))[-1]
                        self.next_state = np.array(self.eng.eval('out.y.Data'))[-1]
#                        self.reward = np.array(self.eng.eval('out.reward.Data'))[-1]

                        return self.clock, self.next_state
                    else:
                        i=i+1
                        if i>10:
                            print('error2');
                            break
            else:
                i=i+1
                if i>10:
                    print('error1');
                    break


if __name__=='__name__':
    simParam={
        'stoptime':10,
        'pause_time':0,
    }

    mandoAgent=MyAgent('pendu',simParam)

    mandoAgent.reset()
    stime=0.02
    num=int(simParam['stoptime']/stime)

    for ii in range(1,num): 
        if ii==1:
            action=0
        else:
            action=-2*y
        t,y=mandoAgent.simstep(action,ii*stime)

    mandoAgent.stop()
